In [ ]:
%load_ext autoreload
%autoreload 2

import cv2
import os
from matplotlib import pyplot as plt
import numpy as np
import lib

PROJECT_ROOT = os.path.abspath(os.path.join(os.curdir, os.pardir))
DATA_ROOT = os.path.join(PROJECT_ROOT, 'data')
EXPORT_ROOT = os.path.join(PROJECT_ROOT, 'export')
os.makedirs(EXPORT_ROOT, exist_ok=True)

PROJECT_ROOT

In [ ]:
CONFIGS = {
    "IMG_20210814_184805.jpg": {
        "objects": [
            {"src_bbox": [[534, 673], [1683, 692], [1681, 1840], [580, 1816]]},
            {"src_bbox": [[1717, 1869], [2853, 1881], [2827, 2977], [1711, 2969]]},
        ]
    },
    "IMG_20210817_203110.jpg": {
        "objects": [
            {"src_bbox": [[1322, 479], [3373, 472], [3387, 2561], [1282, 2507]]}
        ]
    },
    "IMG_20210817_203120.jpg": {
        "objects": [
            {"src_bbox": [[1322, 479], [3373, 472], [3387, 2561], [1282, 2507]]}
        ]
    },
}

IMAGE_FNAME = "IMG_20210817_203110.jpg"

config = CONFIGS[IMAGE_FNAME]
IMAGE_FPATH = os.path.join(DATA_ROOT, "AnalogDevicesImgs", IMAGE_FNAME)


In [ ]:
img = lib.read_image_rgb(IMAGE_FPATH)

# cv2.namedWindow('1', cv2.WINDOW_NORMAL)
# cv2.imshow('1', img)
# cv2.waitKey(0)

In [ ]:


# img = cv2.resize(img, None, fx=.5, fy=.5)

# H_MIN_REL = 0.2
# H_MIN = int(H_MIN_REL*img.shape[0])
# img = img[H_MIN:]

# W_MAX_REL = 0.9
# W_MAX = int(W_MAX_REL*img.shape[1])
# img = img[:, 0:W_MAX]

OBJECTS = config['objects']

DST_SIZE_WH = (400, 400)

TARGET_POINTS = np.array([
    [0, 0],
    [DST_SIZE_WH[0], 0],
    [DST_SIZE_WH[0], DST_SIZE_WH[1]],
    [0, DST_SIZE_WH[1]],
], dtype=np.float32)

for obj in OBJECTS:
    hw_size = np.array(img.shape[:2])

    points = np.array(obj['src_bbox'], dtype=np.float32)

    perspective_mtrx = cv2.getPerspectiveTransform(points, TARGET_POINTS)        
    transformed_img = cv2.warpPerspective(img, perspective_mtrx, DST_SIZE_WH)

    # Export sample
    canvas = img.copy()
    for point in points:
        point = point.astype(int)
        cv2.circle(canvas, tuple(point), radius=50, color=(0, 0, 220), thickness=20)
    lib.write_image_rgb(os.path.join(EXPORT_ROOT, "01_points.png"), canvas)

print(f'Image size: {img.shape}')

lib.write_image_rgb(os.path.join(EXPORT_ROOT, "02_perspective.png"), transformed_img)

###########################################
fig, axs = plt.subplots(nrows=1, ncols=2, figsize=[15,15])

axs[0].imshow(img)
axs[1].imshow(transformed_img)

In [ ]:
eq_transformed_img = lib.equalizeAdaptiveHist(transformed_img, grid_side=5)
# eq_transformed_img = lib.equalizeHist(transformed_img)

thresh_img = lib.binarizeOtsu(eq_transformed_img)

morph_kernel = np.ones((5,5),np.uint8)
morph_img = cv2.morphologyEx(thresh_img, cv2.MORPH_CLOSE, morph_kernel)

###########################################
fig, axs = plt.subplots(nrows=1, ncols=3, figsize=[15,15])

axs[0].imshow(eq_transformed_img)
axs[1].imshow(thresh_img, cmap='gray')
axs[2].imshow(morph_img, cmap='gray')

In [ ]:
bbox_xywh = lib.find_largest_contour(thresh_img)
print(bbox_xywh)

canvas = eq_transformed_img.copy()

x, y, w, h = np.array(bbox_xywh, dtype=int)

cv2.rectangle(canvas, (x, y), (x+w, y+h), color=(0, 0, 220), thickness=10)

###########################################
fig, axs = plt.subplots(nrows=2, ncols=2, figsize=[20,20])

axs[0, 0].imshow(eq_transformed_img)
axs[0, 1].imshow(thresh_img, cmap='gray')
axs[1, 0].imshow(morph_img, cmap='gray')
axs[1, 1].imshow(canvas)

fig.tight_layout()
fig.savefig(os.path.join(EXPORT_ROOT, "03_info_roi.png"))

In [ ]:
screen_roi = lib.extract_subimg_roi(eq_transformed_img, bbox_xywh)

roi_thresh_img = lib.binarizeOtsu(screen_roi, filter_sz=0)

morph = cv2.morphologyEx(np.invert(roi_thresh_img), cv2.MORPH_OPEN, kernel=np.ones((5, 5), dtype=np.uint8))
cleaned_img = np.invert(roi_thresh_img)-morph

THRESHOLD = 90

# Hough (0-45, 135-180)
lines1 = cv2.HoughLines(cleaned_img, rho=1, theta=np.pi / 180, threshold=THRESHOLD, lines=None, srn=0, stn=0, min_theta=0, max_theta=np.pi/4)
lines2 = cv2.HoughLines(cleaned_img, rho=1, theta=np.pi / 180, threshold=THRESHOLD, lines=None, srn=0, stn=0, min_theta=np.pi*3/4, max_theta=np.pi)

lines = []

if lines1 is not None:
    lines.append(lines1)

if lines2 is not None:
    lines.append(lines2)

if len(lines) > 0:
    lines = np.concatenate(lines)

# lines = lib.filter_lines(screen_roi, lines)
screen_canvas = lib.drawLines(screen_roi, lines, color=(0, 0, 255), thickness=1)

if len(lines) > 0:
    rho_line = np.median(np.array(lines)[:,0,0])
    angle_rad = np.median(np.array(lines)[:,0,1])
    angle_rad_std = np.std(np.array(lines)[:,0,1])
    print(rho_line)
    print(np.rad2deg(angle_rad))
    print(np.rad2deg(angle_rad_std))

screen_result_canvas = lib.drawLines(screen_roi, [[[rho_line, angle_rad]]], color=(200, 0, 0), thickness=1)

###########################################
fig, axs = plt.subplots(nrows=3, ncols=2, figsize=[20,25])

axs[0, 0].imshow(screen_roi)
axs[0, 1].imshow(roi_thresh_img, cmap='gray')
axs[1, 0].imshow(morph, cmap='gray')
axs[1, 1].imshow(cleaned_img, cmap='gray')
axs[2, 0].imshow(screen_canvas)
axs[2, 1].imshow(screen_result_canvas)

fig.tight_layout()
fig.savefig(os.path.join(EXPORT_ROOT, "04_arrow.png"))
